In [121]:
import numpy as np
import pandas as pd
import matplotlib as plt
from sklearn import preprocessing
from sklearn.preprocessing import LabelEncoder
from collections import Counter
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import classification_report,confusion_matrix
from sklearn.utils import class_weight
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score
from sklearn.preprocessing import StandardScaler 
%matplotlib inline

In [2]:
train_df = pd.read_csv('data/train.csv')
campaign_df = pd.read_csv('data/campaign_data.csv')

In [3]:
print(train_df.head())

print(campaign_df.head())

          id  user_id  campaign_id         send_date  is_open  is_click
0   42_14051    14051           42  01-09-2017 19:55        0         0
1  52_134438   134438           52  02-11-2017 12:53        0         0
2  33_181789   181789           33  24-07-2017 15:15        0         0
3  44_231448   231448           44  05-09-2017 11:36        0         0
4  29_185580   185580           29  01-07-2017 18:01        0         0
   campaign_id communication_type  total_links  no_of_internal_links  \
0           29         Newsletter           67                    61   
1           30    Upcoming Events           18                    14   
2           31         Conference           15                    13   
3           32         Conference           24                    19   
4           33             Others            7                     3   

   no_of_images  no_of_sections  \
0            12               3   
1             7               1   
2             5               

In [4]:
print(train_df.values.shape)

(1023191, 6)


In [5]:
#result = train_df.join(campaign_df, on='campaign_id')
train = pd.merge(train_df, campaign_df, how='left', on=['campaign_id'])

In [6]:
train.head(10)

,id,user_id,campaign_id,send_date,is_open,is_click,communication_type,total_links,no_of_internal_links,no_of_images,no_of_sections,email_body,subject,email_url
0,42_14051,14051,42,01-09-2017 19:55,0,0,Newsletter,88,79,13,4,"September Newsletter\r\n \r\nDear AVians,\r\n ...",[September] Exciting days ahead with DataHack ...,http://r.newsletters.analyticsvidhya.com/7v3rd...
1,52_134438,134438,52,02-11-2017 12:53,0,0,Newsletter,67,62,10,4,"November Newsletter\r\n \r\nDear AVians,\r\n \...",[Newsletter] Stage for DataHack Summit 2017 is...,http://r.newsletters.analyticsvidhya.com/7vtb2...
2,33_181789,181789,33,24-07-2017 15:15,0,0,Others,7,3,1,1,Fireside Chat with DJ Patil - the master is he...,"[Delhi NCR] Fireside Chat with DJ Patil, Forme...",http://r.newsletters.analyticsvidhya.com/7uvlg...
3,44_231448,231448,44,05-09-2017 11:36,0,0,Upcoming Events,60,56,19,6,"[September Events]\r\n \r\nDear AVians,\r\n \r...","[September] Data Science Hackathons, Meetups a...",http://r.newsletters.analyticsvidhya.com/7veam...
4,29_185580,185580,29,01-07-2017 18:01,0,0,Newsletter,67,61,12,3,"Dear AVians,\r\n \r\nWe are shaping up a super...",Sneak Peek: A look at the emerging data scienc...,http://r.newsletters.analyticsvidhya.com/7um44...
5,49_219916,219916,49,28-09-2017 15:53,0,0,Conference,119,117,16,1,"Dear Patron,\r\n \r\nAnalytics Vidhya is on a ...",Register @ DataHack Summit 2017 - India's Larg...,http://r.newsletters.analyticsvidhya.com/o7ohw...
6,42_177808,177808,42,01-09-2017 20:13,0,0,Newsletter,88,79,13,4,"September Newsletter\r\n \r\nDear AVians,\r\n ...",[September] Exciting days ahead with DataHack ...,http://r.newsletters.analyticsvidhya.com/7v3rd...
7,42_133077,133077,42,01-09-2017 20:11,0,0,Newsletter,88,79,13,4,"September Newsletter\r\n \r\nDear AVians,\r\n ...",[September] Exciting days ahead with DataHack ...,http://r.newsletters.analyticsvidhya.com/7v3rd...
8,49_185813,185813,49,28-09-2017 15:38,0,0,Conference,119,117,16,1,"Dear Patron,\r\n \r\nAnalytics Vidhya is on a ...",Register @ DataHack Summit 2017 - India's Larg...,http://r.newsletters.analyticsvidhya.com/o7ohw...
9,30_153565,153565,30,05-07-2017 14:10,0,0,Upcoming Events,18,14,7,1,"Dear AVians,\r\n \r\nAre your eager to know wh...",[July] Data Science Expert Meetups & Competiti...,http://r.newsletters.analyticsvidhya.com/7up0e...


In [7]:
train = train.drop(['email_url','email_body','subject'],axis=1)


In [16]:
train.head()

,id,send_date,is_open,is_click,communication_type,total_links,no_of_internal_links,no_of_images,no_of_sections
0,42_14051,01-09-2017 19:55,0,0,3,88,79,13,4
1,52_134438,02-11-2017 12:53,0,0,3,67,62,10,4
2,33_181789,24-07-2017 15:15,0,0,4,7,3,1,1
3,44_231448,05-09-2017 11:36,0,0,5,60,56,19,6
4,29_185580,01-07-2017 18:01,0,0,3,67,61,12,3


In [12]:
label_encoder = LabelEncoder()
train['communication_type'] = label_encoder.fit_transform(train['communication_type'])
train.head()

,id,user_id,campaign_id,send_date,is_open,is_click,communication_type,total_links,no_of_internal_links,no_of_images,no_of_sections
0,42_14051,14051,42,01-09-2017 19:55,0,0,3,88,79,13,4
1,52_134438,134438,52,02-11-2017 12:53,0,0,3,67,62,10,4
2,33_181789,181789,33,24-07-2017 15:15,0,0,4,7,3,1,1
3,44_231448,231448,44,05-09-2017 11:36,0,0,5,60,56,19,6
4,29_185580,185580,29,01-07-2017 18:01,0,0,3,67,61,12,3


In [15]:
train= train.drop(['user_id','campaign_id'],axis=1)
train.head()

ValueError: labels ['user_id' 'campaign_id'] not contained in axis

In [69]:
Y = train['is_click'].values
Y.shape
train['is_click'].value_counts()

0    1010409
1      12782
Name: is_click, dtype: int64

In [49]:
X = train[['id','communication_type','total_links','no_of_internal_links','no_of_images','no_of_sections']]
X.shape

(1023191, 6)

In [79]:
X_train, X_test, y_train, y_test = train_test_split(X, Y)

In [80]:
y_train.shape


(767393,)

In [122]:
scaler = StandardScaler()  
scaler.fit(X_train)  
X_train = scaler.transform(X_train)  
# apply same transformation to test data
X_test = scaler.transform(X_test)  

In [123]:
seed = 100
kfold = KFold(n_splits=10, random_state=seed)


model = LogisticRegression(class_weight={0:90,1:10})
model.fit(X,Y)

LogisticRegression(C=1.0, class_weight={0: 90, 1: 10}, dual=False,
          fit_intercept=True, intercept_scaling=1, max_iter=100,
          multi_class='ovr', n_jobs=1, penalty='l2', random_state=None,
          solver='liblinear', tol=0.0001, verbose=0, warm_start=False)

In [133]:
predictions = model.predict(X_test)

print(confusion_matrix(y_test,predictions))
print('classification, roc auc score: %s'
          % roc_auc_score(y_test, predictions))

[[147787 104832]
 [  1879   1300]]
classification, roc auc score: 0.496976482174


In [134]:
mlp = MLPClassifier(activation='relu',hidden_layer_sizes=(8),verbose=True,batch_size=1,random_state=10)

In [135]:
mlp.fit(X_train,y_train)


Iteration 1, loss = 0.06817680
Iteration 2, loss = 0.06767132
Iteration 3, loss = 0.06771267
Iteration 4, loss = 0.06777550
Iteration 5, loss = 0.06778116
Training loss did not improve more than tol=0.000100 for two consecutive epochs. Stopping.


MLPClassifier(activation='relu', alpha=0.0001, batch_size=1, beta_1=0.9,
       beta_2=0.999, early_stopping=False, epsilon=1e-08,
       hidden_layer_sizes=8, learning_rate='constant',
       learning_rate_init=0.001, max_iter=200, momentum=0.9,
       nesterovs_momentum=True, power_t=0.5, random_state=10, shuffle=True,
       solver='adam', tol=0.0001, validation_fraction=0.1, verbose=True,
       warm_start=False)

In [132]:
predictions = mlp.predict(X_test)

print(confusion_matrix(y_test,predictions))
print('classification, roc auc score: %s'
          % roc_auc_score(y_test, predictions))

[[252619      0]
 [  3179      0]]
classification, roc auc score: 0.5


In [28]:
le = preprocessing.LabelEncoder()


# 2/3. FIT AND TRANSFORM
# use df.apply() to apply le.fit_transform to all columns
train_2 = train.apply(le.fit_transform)

In [11]:
train.head()

,id,user_id,campaign_id,send_date,is_open,is_click,communication_type,total_links,no_of_internal_links,no_of_images,no_of_sections,email_body,subject,body_count
0,42_14051,14051,42,01-09-2017 19:55,0,0,Newsletter,88,79,13,4,"September Newsletter\r\n \r\nDear AVians,\r\n ...",[September] Exciting days ahead with DataHack ...,"{'september': 2, 'newsletter': 1, 'dear': 1, '..."
1,52_134438,134438,52,02-11-2017 12:53,0,0,Newsletter,67,62,10,4,"November Newsletter\r\n \r\nDear AVians,\r\n \...",[Newsletter] Stage for DataHack Summit 2017 is...,"{'november': 1, 'newsletter': 1, 'dear': 1, 'a..."
2,33_181789,181789,33,24-07-2017 15:15,0,0,Others,7,3,1,1,Fireside Chat with DJ Patil - the master is he...,"[Delhi NCR] Fireside Chat with DJ Patil, Forme...","{'fireside': 2, 'chat': 2, 'with': 4, 'dj': 3,..."
3,44_231448,231448,44,05-09-2017 11:36,0,0,Upcoming Events,60,56,19,6,"[September Events]\r\n \r\nDear AVians,\r\n \r...","[September] Data Science Hackathons, Meetups a...","{'[september': 1, 'events]': 1, 'dear': 1, 'av..."
4,29_185580,185580,29,01-07-2017 18:01,0,0,Newsletter,67,61,12,3,"Dear AVians,\r\n \r\nWe are shaping up a super...",Sneak Peek: A look at the emerging data scienc...,"{'dear': 1, 'avians,': 1, 'we': 3, 'are': 4, '..."
